**Import Package**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Collecting tweets and news exerpts: Sentiment and Emotion Analysis

In [ ]:
# conducting sentiment analysis
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# downloading lexicon
nltk.download('vader_lexicon')

# instantiating model
sia = SentimentIntensityAnalyzer()


In [ ]:
def sentiment_analysis(tweet_text): # need to add emotional score into the list
    score = sia.polarity_scores(tweet_text)["compound"]
    return score

In [ ]:
# conducting emotional analysis

#import requests
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions

base_url = 'insert credentials'
api_key = 'insert credentials'

authenticator = IAMAuthenticator(api_key)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2022-04-07',
    authenticator=authenticator)

natural_language_understanding.set_service_url(base_url)

In [ ]:
def emotional_analysis(text_content):
    response = natural_language_understanding.analyze(text=text_content, features=Features(emotion=EmotionOptions(document=True)), language='en').get_result()
    res = []
    # output = json.dumps(response, indent=2)
    emotions = response['emotion']['document']['emotion']
    for key, value in emotions.items():
        if key == 'sadness':
            sadness_score = value
        if key == 'joy':
            joy_score = value
        if key == 'fear':
            fear_score = value
        if key == 'disgust':
            disgust_score = value
        if key == 'anger':
            anger_score = value
    res = [sadness_score, joy_score, fear_score, disgust_score, anger_score]
    return res

In [ ]:
# connecting to twitter
import tweepy, pandas

# creating the 4 keys 
consumer_key = 'insert credentials'
consumer_secret = 'insert credentials'
access_token = 'insert credentials'
access_token_secret = 'insert credentials'

# authenticating
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
# import the required modules
from time import sleep

# define relevant variables
# since there were limitations in how many tweets I can collect at once, I only collect up to 1000 tweets everyday, eliminating the retweets
seen = set() # this set collect tweet id to verify if a tweet was collected
tweet_df = pandas.DataFrame(columns = ["Tweet's Text", "Sentiment Score", "Sadness Score", "Joy Score", "Fear Score", \
    "Disgust Score", "Anger Score", "Hashtags", "Created At"])

while True:
    # retrieve five tweets
    for tweet in tweepy.Cursor(api.search_tweets, q="oil price OR gas price", lang="en", since_id = "2022-05-04", until="2022-05-04").items(1000):
        # verify whether tweet was not collected previously or if the tweet is a retweet
        if (tweet.id not in seen) and ('RT @' not in tweet.text):
            seen.add(tweet.id)
            # calculate the sentiment score associated with tweet
            hashtag = tweet.entities["hashtags"]
            sentiment_score = sentiment_analysis(tweet.text)
            sadness_score, joy_score, fear_score, disgust_score, anger_score = emotional_analysis(tweet.text)
            # add to current df
            ls = [tweet.text, sentiment_score, sadness_score, joy_score, fear_score, disgust_score, anger_score, [hashtag if hashtag else None], tweet.created_at]
            tweet_df.loc[len(tweet_df)] = ls

    # # print current cumulative_score
    print(tweet_df)
    # sleeping for 10 seconds to avoid too many Twitter queries
    sleep(10)

In [ ]:
# export tweet dataframe daily
tweet_df.to_csv("tweet_df_0504.csv")

In [ ]:
# pull data from news article using newscatcherapi
import requests
import json
import time
import datetime
import pandas as pd

# Put your API key to headers in order to be authorized to perform a call
headers = {'x-api-key': 'insert credentials'}
url = "https://api.newscatcherapi.com/v2/search"
start_date = datetime.date(2022, 4, 1)
end_date = datetime.date(2022, 4, 29)
delta = datetime.timedelta(days=1)

while start_date <= end_date:

    # Define your desired parameters
    params = {
        'q': 'oil price OR gas price',
        'lang': 'en',
        'to_rank': 10000,
        'page_size': 100,
        'page': 1,
        'from': start_date,
        'to': start_date
        }

    # Make a simple call with both headers and params
    response = requests.get(url, headers=headers, params=params)

    # Encode received results
    article_results = json.loads(response.text.encode())
    if response.status_code == 200:
        print('Done')
    else:
        print(article_results)
        print('ERROR: API call failed.')
    # add to current data frame
    news_data = pd.DataFrame(article_results['articles'])
    all_news_data = all_news_data.append(news_data)

    start_date = start_date + delta
    time.sleep(5)


In [ ]:
# Using lambda function to apply sentiment score and emotional score. All NAs were removed in csv files and imported back to python
all_news_data['sentiment_score'] = all_news_data.apply(lambda row : sentiment_analysis(row[8]), axis=1)
all_news_data['emotional_score'] = all_news_data.apply(lambda row : emotional_analysis(row[8]), axis=1)
all_news_data[['sadness_score', 'joy_score', 'fear_score', 'disgust_score', 'anger_score']] = pd.DataFrame(all_news_data.emotional_score.tolist(), index= all_news_data.index)

In [ ]:
# exporting news data 
all_news_data.to_csv('news_article_april.csv')
## note: since there's a time limitation in how much data can be collected through this api (30 days into the past), news data for april and may are saved in different files

**Granger Test on Tweets and News Articles**

In [ ]:
tweets_sum_scores = pd.read_csv('tweets_sum_scores_noNA.csv')
sadness_score = tweets_sum_scores['sadness_score'].to_list()
oil_price = tweets_sum_scores['WTIOIL'].to_list()

# news_article_sum_scores = pd.read_csv('news_article_sum_scores_noNA.csv') # use this line to read in news articles data

sadness_score = tweets_sum_scores['sadness_score'].to_list() # replace the tweets_sum_scores with the news_article_sum_scores to run test on news data
oil_price = tweets_sum_scores['WTIOIL'].to_list()

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 2 plots were created to showcase different time series vs oil time series data as sentiment and emotions scores are much smaller
# first time series
plt.figure(figsize=(10,4))
plt.plot(sadness_score, color='b')

plt.legend(['sadness_score'], fontsize=16)

In [ ]:
# second time series
plt.figure(figsize=(10,4))
# plt.plot(fear_score, color='b')
plt.plot(oil_price, color='r')

plt.legend(['oil_price'], fontsize=16)

In [ ]:
# perform Granger on dataset
ts_df = pd.DataFrame(columns=['oil_price', 'sadness_score'], data=zip(oil_price, sadness_score))
ts_df = ts_df.iloc[:-2] # eliminate the last 2 row, one without oil price data and a blank row that was imported
gc_res = grangercausalitytests(ts_df, 1)

## Analyzing the relationship between stock market and oil price

In [ ]:
# importing stock data from yfinance
import yfinance as yf
import pandas as pd

df_energy = yf.download('^GSPE', '2018-01-06', interval = '1wk')
df_healthcare = yf.download('^SP500-35', '2018-01-06', interval = '1wk')
df_realestate = yf.download('^SP500-60', '2018-01-06', interval = '1wk')
df_material = yf.download('^SP500-15', '2018-01-06', interval = '1wk')
df_industrials = yf.download('^SP500-20', '2018-01-06', interval = '1wk')
df_financials = yf.download('^SP500-40', '2018-01-06', interval = '1wk')
df_utilities = yf.download('^SP500-55', '2018-01-06', interval = '1wk')
df_info_technology = yf.download('^SP500-45', '2018-01-06', interval = '1wk')
df_communication = yf.download('^SP500-50', '2018-01-06', interval = '1wk')
df_consumer_discretionary = yf.download('^SP500-25', '2018-01-06', interval = '1wk')
df_consumer_staples = yf.download('^SP500-30', '2018-01-06', interval = '1wk')
oil_price = pd.read_csv('WCOILWTICO.csv')

In [ ]:
# subsetting the dataframes; will only be looking at the Adj Close price
df_energy = df_energy[['Adj Close']]
df_healthcare = df_healthcare[['Adj Close']]
df_realestate = df_realestate[['Adj Close']]
df_material = df_material[['Adj Close']]
df_industrials = df_industrials[['Adj Close']]
df_financials = df_financials[['Adj Close']]
df_utilities = df_utilities[['Adj Close']]
df_info_technology = df_info_technology[['Adj Close']]
df_communication = df_communication[['Adj Close']]
df_consumer_discretionary = df_consumer_discretionary[['Adj Close']]
df_consumer_staples = df_consumer_staples[['Adj Close']]

In [ ]:
# rename column for the stocks
df_energy = df_energy.rename(columns={'Adj Close': 'Energy'})
df_healthcare = df_healthcare.rename(columns={'Adj Close': 'Healthcare'})
df_realestate = df_realestate.rename(columns={'Adj Close': 'Real Estate'})
df_material = df_material.rename(columns={'Adj Close': 'Material'})
df_industrials = df_industrials.rename(columns={'Adj Close': 'Industrials'})
df_financials = df_financials.rename(columns={'Adj Close': 'Financials'})
df_utilities = df_utilities.rename(columns={'Adj Close': 'Utilities'})
df_info_technology = df_info_technology.rename(columns={'Adj Close': 'Info Tech'})
df_communication = df_communication.rename(columns={'Adj Close': 'Communication'})
df_consumer_discretionary = df_consumer_discretionary.rename(columns={'Adj Close': 'Consumer Discretionary'})
df_consumer_staples = df_consumer_staples.rename(columns={'Adj Close': 'Consumer Staples'})


In [ ]:
# merge stock df
merged_stock_df = pd.merge(df_energy, df_healthcare, left_index=True, right_index=True, how='left')
merged_stock_df1 = pd.merge(df_realestate, df_material, left_index=True, right_index=True, how='left')
merged_stock_df2 = pd.merge(df_industrials, df_financials, left_index=True, right_index=True, how='left')
merged_stock_df3 = pd.merge(df_utilities, df_info_technology, left_index=True, right_index=True, how='left')
merged_stock_df4 = pd.merge(df_communication, df_consumer_discretionary, left_index=True, right_index=True, how='left')
# second merge
merged_stock_df = pd.merge(merged_stock_df, merged_stock_df1, left_index=True, right_index=True, how='left')
merged_stock_df2 = pd.merge(merged_stock_df2, merged_stock_df3, left_index=True, right_index=True, how='left')
merged_stock_df4 = pd.merge(merged_stock_df4, df_consumer_staples, left_index=True, right_index=True, how='left')
# third merge
merged_stock_df = pd.merge(merged_stock_df, merged_stock_df2, left_index=True, right_index=True, how='left')
merged_stock_df = pd.merge(merged_stock_df, merged_stock_df4, left_index=True, right_index=True, how='left')
# export the dataframe
merged_stock_df.to_csv("merged_stock_df.csv")

In [ ]:
# create correlation matrix between sector index and oil price
import seaborn as sns

import matplotlib.pyplot as plt

stock_df = pd.read_csv('merged_stock_df.csv')
# stock_df = stock_df.iloc[:-123] # run this line of code to get data from 2018-2019 only

hm = sns.heatmap(stock_df.corr(), annot = True)

hm.set(title = "Correlation matrix of Sector Index with Oil Price\n")

plt.show()

In [ ]:
# perform Granger test on the ones with high correlations, except Energy 
oil_price = stock_df['WCOILWTICO'].to_list()
financials = stock_df['Financials'].to_list()

# run these codes to get data for Real Estate sector
# real_estate = stock_df['Real Estate'].to_list()
# real_estate = [x for x in real_estate if pd.isnull(x) == False]
# run these codes to get data for Utilities sector
# real_estate = stock_df['Real Estate'].to_list()

# run these codes to perform granger test for stock related indices
ts2_df = pd.DataFrame(columns=['oil_price', 'financials'], data=zip(oil_price, financials))
ts2_df = ts2_df.iloc[:-2]
gc_res2 = grangercausalitytests(ts2_df, 4)

**Fear Index vs Oil Price**

In [ ]:
# gathering data for VIX
import yfinance as yf
import pandas as pd

df_vix = yf.download('^VIX', '2018-01-06', interval = '1wk')
df_vix = df_vix[['Adj Close']]
df_vix = df_vix.rename(columns={'Adj Close': '^VIX'})
df_vix.to_csv("df_vix.csv")
# merge df_vix with oil price on excel

In [ ]:
#see causality between vix causing oilprice from 2018-2022
df_vix = pd.read_csv('df_vix.csv')
vix_score = df_vix['^VIX'].to_list()
oil_price = df_vix['WCOILWTICO'].to_list()

#see causality between vix causing oilprice from 2018-2019
# df_vix2 = pd.read_csv('df_vix2.csv')
# vix_score = df_vix2['^VIX'].to_list()
# oil_price = df_vix2['WCOILWTICO'].to_list()

from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# first time series, news sentiments
plt.figure(figsize=(10,4))
plt.plot(vix_score, color='b')
plt.plot(oil_price, color='r')

plt.legend(['vix_index', 'oil_price'], fontsize=16)

In [ ]:
ts1_df = pd.DataFrame(columns=['oil_price', 'vix_score'], data=zip(oil_price, vix_score))
ts1_df = ts1_df.iloc[:-2]
gc_res1 = grangercausalitytests(ts1_df, 3)